In [34]:
import numpy as np
from time import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#import Noise

%run Noise.ipynb

In [60]:
def general_1d_solver(L, u0, W, mu, sigma = lambda x: 1, T = 1, X = 1, Burgers = 0, compl = False):
    
    # L = [c_0, ... c_3, c_4] corresponds to a differential operator L = \sum c_i \partial^i_x
    # in the parabolic operator (\partial_t - L)
    
    # NB: including c_0 != 0 allows to ignore it in the non-linearity: taking mu(u) = -u^3 + 3u and c_0 = 0 
    # is equivalent to taking mu(u) = -u^3 and c_0 = 3 
    
    
    
    # u0 - initial condition
    # xi - space time noise
    # mu - drift term of the equation
    # sigma - diffusion term of the equation
    # T - time horizon
    # X - space domain is [0,X]
    # Burgers - coefficient in front of the Burger's term u*\partial_x u in the equation
    
    # Check operator conditoins. Must have c_4 < 0 or c_2 > 0 if c_4 = 0
    if (type(L[-1]) != complex) and (L[-1] > 0) or (L[-1] == 0 and type(L[2]) != complex and L[2] < 0):
        print("Differential Operator is not Elliptic.")
        return
    
    # space time grid
    M, N =  W.shape[-1], W.shape[-2] - 1,
    dt = T/N
    
    
    # Time deirvative of the noise
    
    if len(W.shape) > 2:
        dW = np.zeros(W.shape)
        dW[:,1:,:] = np.diff(W, axis = 1)
    else:
        dW = np.zeros((1, W.shape[0], W.shape[1]))
        dW[:,1:,:] = np.diff(W, axis = 0)
    
    K = np.arange(M) # Fourier space
    # Space derivatives in Fourier space. These are analogues of central finite difference in Fourier Space
    # All these give an error of O(N^{-2}) if n-th derivative is approximated by (2*np.pi*K)^n
    
    dx = M/X*1j*np.sin(2*np.pi*K/M)
    d2x = 2*(M/X)**2*(np.cos(2*np.pi*K/M) - 1)
    d3x = dx*d2x
    d4x = 2*(M/X)**4*(np.cos(4*np.pi*K/M) - 4*np.cos(2*np.pi*K/M) + 3)
    
    Lk = L[0] + L[1]*dx + L[2]*d2x + L[3]*d3x + L[4]*d4x # Full differential operator in the Fourier space 
    
    if compl:
        soln = np.zeros(dW.shape) + 1j*np.zeros(dW.shape)
    else:
        soln = np.zeros(dW.shape)
    
    soln[:,0,:] = u0 # set the initial condition
    
    w = np.fft.fft(u0, axis = -1)
    
    for i in tqdm(range(1, N+1)):
        
        if Burgers != 0: # if Burgers term is present compute space derivative. M/X = (dx)^{-1}
            diff = np.zeros((soln.shape[0], M))
            diff[:, 1:] = np.diff(soln[:,i-1,:], axis = 1)
            diff[:, 0] = soln[:,i-1,0] - soln[:,i-1,-1]
            B = Burgers*soln[:,i-1,:]*diff*M/X
        else:
            B = 0
        
        # non-linearities in the physical space
        RHS = np.vectorize(mu)(soln[:,i-1,:])*dt + np.vectorize(sigma)(soln[:,i-1,:])*dW[:,i,:] + B*dt
        
        # updating the Fourier transform for the next time step
        w = ( (1+0.5*Lk*dt) * w + np.fft.fft(RHS, axis = -1) )/(1-0.5*Lk*dt)
        
        # Going back to the physiscal space
        if compl:
            soln[:,i,:] = np.fft.ifft(w, axis = -1)
        else:
            soln[:,i,:] = np.fft.ifft(w, axis = -1).real
        
        
    return soln, np.linspace(0,T, N+1), np.linspace(0, X, M+1)[:-1]    

In [53]:
N = 1 # number of realizations

# time space boundaries
s, t, a, b = 0, 1, 0, 1

# time space mesh sizes
dt, dx = 1e-3, 0.005

# Generate N realizations of space time white noise. 
# Might take a while if the dt = 1e-4. Around 441 seconds to generate 1000 noises with mesh dt = 1e-4 on my laptop
# and 146 seconds with dt = 1e-3.

s_ = -time()
W = Noise().WN_space_time_many(s, t, dt, a, b, dx, N)
print(s_+time())

#space time domain
T, X = np.linspace(0,1, W.shape[1]), np.linspace(0,1, W.shape[2])

# generate N realizations of initial conditions like in the paper
u0 = np.array([[x*(1-x) for x in X[:-1]] for _ in range(N)])
lambd = 0.1 # strength of the randomness

u0 = u0 + lambd*Noise().initial(N, X, p = 10)[:,:-1] # Delete the last space point which is equal to the first one

mu = lambda x: -x**3 + 3*x
mu_ = lambda x: -x**3 # the above mu but without the linear part which could be absorbed in the linear operator
sigma = lambda x: x

# Differential operator

L = [3,-1,1,3,-0.5]

# Nonlinear Schrodinger equation 

mu_s = lambda x : -1j*x*np.absolute(x)**2
L_s = [0,0,1j,0,0]

0.1501760482788086


In [47]:
# Might be worth spliting into batches of 100 noises and apply general_1d_solver separately if dt = 1e-4. In general_1d_solver
# noise W is transformed into dW which might take a while if N is large and dt is small.
soln, _, _ = general_1d_solver(L, u0, W[:,:,:-1], mu = mu_, sigma = sigma)

# Ignore the last space point in the noise which is equal to the first one by periodicity in W[:,:,:-1]

100%|██████████| 1000/1000 [00:00<00:00, 5715.60it/s]


In [61]:
Schrodinger, _, _ = general_1d_solver(L_s, u0, W[:,:,:-1], mu = mu_s, sigma = lambda x : 0, compl = True)

100%|██████████| 1000/1000 [00:00<00:00, 1977.16it/s]
